In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
import optuna
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from lazypredict.Supervised import LazyClassifier
import lightgbm as lgbm
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import  train_test_split

In [2]:
SPACE_PATH='datasets/Spaceship'

In [3]:
def load_housing_data(data_csv,housing_path=SPACE_PATH):
    csv_path=os.path.join(housing_path,data_csv)
    return (pd.read_csv(csv_path))

In [4]:
train0=load_housing_data("train.csv")
test0=load_housing_data("test.csv")

In [5]:
train0.head(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.00,False,0.00,0.00,0.00,0.00,0.00,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.00,False,109.00,9.00,25.00,549.00,44.00,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.00,True,43.00,3576.00,0.00,6715.00,49.00,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.00,False,0.00,1283.00,371.00,3329.00,193.00,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.00,False,303.00,70.00,151.00,565.00,2.00,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.00,False,0.00,483.00,0.00,291.00,0.00,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.00,False,42.00,1539.00,3.00,0.00,0.00,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.00,False,0.00,0.00,0.00,0.00,NaN,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.00,False,0.00,785.00,17.00,216.00,0.00,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.00,False,0.00,0.00,0.00,0.00,0.00,Erraiam Flatic,True


In [6]:
test0.head(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.00,False,0.00,0.00,0.00,0.00,0.00,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.00,False,0.00,9.00,0.00,2823.00,0.00,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.00,False,0.00,0.00,0.00,0.00,0.00,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.00,False,0.00,6652.00,0.00,181.00,585.00,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.00,False,10.00,0.00,635.00,0.00,0.00,Brence Harperez
5,0027_01,Earth,False,F/7/P,TRAPPIST-1e,31.00,False,0.00,1615.00,263.00,113.00,60.00,Karlen Ricks
6,0029_01,Europa,True,B/2/P,55 Cancri e,21.00,False,0.00,NaN,0.00,0.00,0.00,Aldah Ainserfle
7,0032_01,Europa,True,D/0/S,TRAPPIST-1e,20.00,False,0.00,0.00,0.00,0.00,0.00,Acrabi Pringry
8,0032_02,Europa,True,D/0/S,55 Cancri e,23.00,False,0.00,0.00,0.00,0.00,0.00,Dhena Pringry
9,0033_01,Earth,False,F/7/S,55 Cancri e,24.00,False,0.00,639.00,0.00,0.00,0.00,Eliana Delazarson


In [7]:
print("Shape of the train set : ",train0.shape)
print("Sahpe of the test set : ",test0.shape)

Shape of the train set :  (8693, 14)
Sahpe of the test set :  (4277, 13)


In [8]:
len(train0.columns)

14

In [9]:
train0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [10]:
test0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [11]:
test_Id=test0['PassengerId']
target=train0['Transported']

In [12]:
train1=train0.drop(["PassengerId","Transported"],axis=1)
test1=test0.drop(['PassengerId'],axis=1)

In [13]:
print("Duplicates in Train Dataset is:",train1.duplicated().sum()/len(train1),"%")
print("Duplicates in Test Dataset is:",test1.duplicated().sum()/(len(test1)),"%")

Duplicates in Train Dataset is: 0.0 %
Duplicates in Test Dataset is: 0.0 %


In [14]:
data=pd.concat([train1,test1],axis=0).reset_index(drop=True)

In [15]:
data.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,12700.00,12707.00,12681.00,12664.00,12686.00,12702.00
mean,28.77,222.90,451.96,174.91,308.48,306.79
std,14.39,647.60,1584.37,590.56,1130.28,1180.10
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,19.00,0.00,0.00,0.00,0.00,0.00
50%,27.00,0.00,0.00,0.00,0.00,0.00
75%,38.00,49.00,77.00,29.00,57.00,42.00
max,79.00,14327.00,29813.00,23492.00,22408.00,24133.00


In [16]:
df = (data.isnull().sum()[data.isnull().sum()>0]).to_frame().rename(columns={0:"Number of Missing values"})
df["% of Missing Values"] = round((100*data.isnull().sum()[data.isnull().sum()>0]/len(data)),2)
df

,Number of Missing values,% of Missing Values
HomePlanet,288,2.22
CryoSleep,310,2.39
Cabin,299,2.31
Destination,274,2.11
Age,270,2.08
VIP,296,2.28
RoomService,263,2.03
FoodCourt,289,2.23
ShoppingMall,306,2.36
Spa,284,2.19


In [17]:
print("cardinality of categorical features in  datasets is:")
print(data.select_dtypes(include="object").nunique())

cardinality of categorical features in  datasets is:
HomePlanet         3
CryoSleep          2
Cabin           9825
Destination        3
VIP                2
Name           12629
dtype: int64


In [18]:
data1=data.copy()

In [19]:
data1=data1.drop(['Name'],axis=1)

In [20]:
data2=data1.copy()

In [21]:
data3=data2.copy()

In [22]:
(data3.select_dtypes("object").isna()==True).columns

Index(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP'], dtype='object')

In [23]:
mis_cat_columns=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']

In [24]:
#impute categorical variables that are not boolean
for column in mis_cat_columns:
    data3[column]=data3[column].fillna(data3[column].mode()[0])

In [25]:
#impute numerical variables that are boolean
num_columns=data3.select_dtypes("float64").columns
num_columns

Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')

In [26]:
mis_num_columns=(data3.select_dtypes("float64").isna()==True).columns
mis_num_columns

Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')

In [27]:
for column in mis_num_columns:
    data3[column]=data3[column].fillna(data3[column].mean())

In [28]:
data3.isna().sum()

HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64

In [29]:
data4=data3.copy()

In [30]:
skew_df = pd.DataFrame(data4.select_dtypes(np.number).columns, columns=['Feature'])
skew_df['Skew'] = skew_df['Feature'].apply(lambda feature: scipy.stats.skew(data4[feature]))
skew_df['Absolute Skew'] = skew_df['Skew'].apply(abs)
skew_df['Skewed'] = skew_df['Absolute Skew'].apply(lambda x: True if x >= 0.5 else False)
skew_df

,Feature,Skew,Absolute Skew,Skewed
0,Age,0.44,0.44,False
1,RoomService,6.20,6.20,True
2,FoodCourt,7.13,7.13,True
3,ShoppingMall,11.14,11.14,True
4,Spa,7.74,7.74,True
5,VRDeck,8.14,8.14,True


In [31]:
for column in skew_df.query("Skewed == True")['Feature'].values:
    data4[column] = np.log1p(data4[column])

In [32]:
data5=data4.copy()

In [33]:
data5 = pd.get_dummies(data5)
data6 = data5.copy()

In [34]:
scaler = StandardScaler()
scaler.fit(data6)

data7 = pd.DataFrame(scaler.transform(data6), index=data6.index, columns=data6.columns)

In [35]:
data8 = data7.copy()

In [36]:
data8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Columns: 9839 entries, CryoSleep to Destination_TRAPPIST-1e
dtypes: float64(9839)
memory usage: 973.6 MB


In [55]:
train_final = data8.loc[:train0.index.max(), :]

In [56]:
train_final

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,...,Cabin_T/1/P,Cabin_T/1/S,Cabin_T/2/P,Cabin_T/2/S,Cabin_T/3/P,Cabin_T/3/S,Cabin_T/4/P,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,-0.74,0.72,-0.15,-0.67,-0.68,-0.66,-0.70,-0.67,-1.11,1.77,...,-0.01,-0.01,-0.01,-0.01,-0.01,-0.02,-0.01,-0.51,-0.32,0.65
1,-0.74,-0.34,-0.15,1.03,0.09,0.58,1.55,0.70,0.90,-0.56,...,-0.01,-0.01,-0.01,-0.01,-0.01,-0.02,-0.01,-0.51,-0.32,0.65
2,-0.74,2.05,6.82,0.70,2.06,-0.66,2.44,0.74,-1.11,1.77,...,-0.01,-0.01,-0.01,-0.01,-0.01,-0.02,-0.01,-0.51,-0.32,0.65
3,-0.74,0.30,-0.15,-0.67,1.72,1.60,2.19,1.23,-1.11,1.77,...,-0.01,-0.01,-0.01,-0.01,-0.01,-0.02,-0.01,-0.51,-0.32,0.65
4,-0.74,-0.90,-0.15,1.40,0.75,1.26,1.56,-0.27,0.90,-0.56,...,-0.01,-0.01,-0.01,-0.01,-0.01,-0.02,-0.01,-0.51,-0.32,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,-0.74,0.86,6.82,-0.67,2.28,-0.66,1.94,0.88,-1.11,1.77,...,-0.01,-0.01,-0.01,-0.01,-0.01,-0.02,-0.01,1.98,-0.32,-1.55
8689,1.35,-0.76,-0.15,-0.67,-0.68,-0.66,-0.70,-0.67,0.90,-0.56,...,-0.01,-0.01,-0.01,-0.01,-0.01,-0.02,-0.01,-0.51,3.16,-1.55
8690,-0.74,-0.19,-0.15,-0.67,-0.68,2.22,-0.45,-0.67,0.90,-0.56,...,-0.01,-0.01,-0.01,-0.01,-0.01,-0.02,-0.01,-0.51,-0.32,0.65
8691,-0.74,0.23,-0.15,-0.67,1.65,-0.66,1.39,2.24,-1.11,1.77,...,-0.01,-0.01,-0.01,-0.01,-0.01,-0.02,-0.01,1.98,-0.32,-1.55


In [39]:
data8.isna().sum()

CryoSleep                    0
Age                          0
VIP                          0
RoomService                  0
FoodCourt                    0
                            ..
Cabin_T/3/S                  0
Cabin_T/4/P                  0
Destination_55 Cancri e      0
Destination_PSO J318.5-22    0
Destination_TRAPPIST-1e      0
Length: 9839, dtype: int64

In [59]:
test_final=data8.loc[train0.index.max()+1:,:]

In [41]:
X_train , X_test , y_train , y_test = train_test_split(train_final, target, stratify = target,
                                                       random_state = 5, test_size = 0.40)

In [42]:
"""clf = LazyClassifier(verbose = 0,
                     ignore_warnings = True,
                     custom_metric = None,
                     predictions = False,
                     random_state = 12,
                     classifiers = 'all')

models, predictions = clf.fit(X_train, X_test,y_train,y_test)"""

"clf = LazyClassifier(verbose = 0,\n                     ignore_warnings = True,\n                     custom_metric = None,\n                     predictions = False,\n                     random_state = 12,\n                     classifiers = 'all')\n\nmodels, predictions = clf.fit(X_train, X_test,y_train,y_test)"

In [43]:
from optuna.integration import LightGBMPruningCallback

def objective(trial, X, y):
    param_grid = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = lgbm.LGBMClassifier(objective="binary", **param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="binary_logloss",
            early_stopping_rounds=100,
            callbacks=[
                LightGBMPruningCallback(trial, "binary_logloss")
            ],  # Add a pruning callback
        )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)

In [44]:
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, train_final, target)
study.optimize(func, n_trials=20)

[I 2023-02-18 21:49:40,151] A new study created in memory with name: LGBM Classifier


[LightGBM] [Warning] feature_fraction is set=0.30000000000000004, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.30000000000000004
[LightGBM] [Warning] min_data_in_leaf is set=8900, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8900
[LightGBM] [Warning] min_gain_to_split is set=9.23319308496458, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=9.23319308496458
[LightGBM] [Warning] lambda_l1 is set=20, reg_alpha=0.0 will be ignored. Current value: lambda_l1=20
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] lambda_l2 is set=40, reg_lambda=0.0 will be ignored. Current value: lambda_l2=40
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[1]	valid_0's binary_logloss: 0.693143
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.693143


[I 2023-02-18 21:51:45,203] Trial 0 finished with value: 0.6931209418831152 and parameters: {'n_estimators': 10000, 'learning_rate': 0.27506305725430064, 'num_leaves': 1800, 'max_depth': 9, 'min_data_in_leaf': 8900, 'lambda_l1': 20, 'lambda_l2': 40, 'min_gain_to_split': 9.23319308496458, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 0 with value: 0.6931209418831152.


[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=9300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9300
[LightGBM] [Warning] min_gain_to_split is set=3.9098083293928765, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=3.9098083293928765
[LightGBM] [Warning] lambda_l1 is set=55, reg_alpha=0.0 will be ignored. Current value: lambda_l1=55
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] lambda_l2 is set=75, reg_lambda=0.0 will be ignored. Current value: lambda_l2=75
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[1]	valid_0's binary_logloss: 0.693143
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.693143
[3]	valid_0's binary_logloss

[I 2023-02-18 21:53:18,526] Trial 1 finished with value: 0.6931209418831152 and parameters: {'n_estimators': 10000, 'learning_rate': 0.02597200255417309, 'num_leaves': 1560, 'max_depth': 10, 'min_data_in_leaf': 9300, 'lambda_l1': 55, 'lambda_l2': 75, 'min_gain_to_split': 3.9098083293928765, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.7}. Best is trial 0 with value: 0.6931209418831152.


...

[I 2023-02-18 22:10:16,837] Trial 17 finished with value: 0.6931209418831152 and parameters: {'n_estimators': 10000, 'learning_rate': 0.1644604651382456, 'num_leaves': 420, 'max_depth': 12, 'min_data_in_leaf': 1100, 'lambda_l1': 75, 'lambda_l2': 10, 'min_gain_to_split': 2.0482730343059807, 'bagging_fraction': 0.2, 'bagging_freq': 1, 'feature_fraction': 0.8}. Best is trial 10 with value: 0.4520484207084059.


[LightGBM] [Warning] feature_fraction is set=0.30000000000000004, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.30000000000000004
[LightGBM] [Warning] min_data_in_leaf is set=2900, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=2900
[LightGBM] [Warning] min_gain_to_split is set=8.313805189201778, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=8.313805189201778
[LightGBM] [Warning] lambda_l1 is set=15, reg_alpha=0.0 will be ignored. Current value: lambda_l1=15
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] lambda_l2 is set=55, reg_lambda=0.0 will be ignored. Current value: lambda_l2=55
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[1]	valid_0's binary_logloss: 0.693143
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.69314

[I 2023-02-18 22:11:18,123] Trial 18 finished with value: 0.6931209418831152 and parameters: {'n_estimators': 10000, 'learning_rate': 0.12075940150955602, 'num_leaves': 1300, 'max_depth': 3, 'min_data_in_leaf': 2900, 'lambda_l1': 15, 'lambda_l2': 55, 'min_gain_to_split': 8.313805189201778, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 10 with value: 0.4520484207084059.


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=2500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=2500
[LightGBM] [Warning] min_gain_to_split is set=5.815211292623731, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=5.815211292623731
[LightGBM] [Warning] lambda_l1 is set=30, reg_alpha=0.0 will be ignored. Current value: lambda_l1=30
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] lambda_l2 is set=30, reg_lambda=0.0 will be ignored. Current value: lambda_l2=30
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[1]	valid_0's binary_logloss: 0.693143
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.693143
[3]	valid_0's binary_logloss: 

[I 2023-02-18 22:12:16,813] Trial 19 finished with value: 0.6931209418831152 and parameters: {'n_estimators': 10000, 'learning_rate': 0.29982777632004937, 'num_leaves': 2120, 'max_depth': 5, 'min_data_in_leaf': 2500, 'lambda_l1': 30, 'lambda_l2': 30, 'min_gain_to_split': 5.815211292623731, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.8}. Best is trial 10 with value: 0.4520484207084059.


In [45]:
study.best_params

{'n_estimators': 10000,
 'learning_rate': 0.13415755110594887,
 'num_leaves': 1140,
 'max_depth': 3,
 'min_data_in_leaf': 200,
 'lambda_l1': 0,
 'lambda_l2': 5,
 'min_gain_to_split': 1.2179833771492579,
 'bagging_fraction': 0.5,
 'bagging_freq': 1,
 'feature_fraction': 0.2}

In [47]:
best_paramas={'n_estimators': 10000,
              'learning_rate': 0.13415755110594887,
              'num_leaves': 1140,
              'max_depth': 3,
              'min_data_in_leaf': 200,
              'lambda_l1': 0,
              'lambda_l2': 5,
              'min_gain_to_split': 1.2179833771492579,
              'bagging_fraction': 0.5,
              'bagging_freq': 1,
              'feature_fraction': 0.2}

In [50]:
model = lgbm.LGBMClassifier(objective="binary", **best_paramas)
model.fit(
    train_final,target
   # Add a pruning callback
)

LGBMClassifier(bagging_fraction=0.5, bagging_freq=1, feature_fraction=0.2,
               lambda_l1=0, lambda_l2=5, learning_rate=0.13415755110594887,
               max_depth=3, min_data_in_leaf=200,
               min_gain_to_split=1.2179833771492579, n_estimators=10000,
               num_leaves=1140, objective='binary')

In [62]:
final_predictions=model.predict(test_final)
submission = pd.concat([test_Id, pd.Series(final_predictions.astype("bool"), name='Transported')], axis=1)

In [63]:
submission.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [64]:
submission.to_csv('C:/Users/hp/Desktop/Projects/datasets/Spaceship/submission.csv', index=False, header=True)